In [ ]:
# PVOL R CODE
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - local_pvol_paths : List
#   outputs:
#    - local_ppi_paths : List
#   params:
#    - param_param : String
#    - param_elevation : Float
# ...




# Code analyzer fix block. Assign empty strings to variables that should not be picked up by the analyzer.
pvol_path = ""
im_fname = ""
filename_noext = ""
filename_noext_parts = ""
filename = ""
local_image_path = ""

# Print conf to ensure conf passing to cell build
print(conf_local_visualization_output)

# Libraries
library('bioRad')
library('tools')
library('ggplot2')

# Functions
im_fname_from_path <- function(path,elev,param,imtype = 'ppi',im_ext="png"){
  filename <- basename(path)
  filename_noext <- tools::file_path_sans_ext(filename)
  filename_noext_parts <- unlist(strsplit(filename_noext,split="_"))
  im_fname<-paste0(
    c(filename_noext_parts[[1]],imtype,gsub(".","-",elev,fixed=T),param,filename_noext_parts[[3]],
      paste0(c(filename_noext_parts[[4]],im_ext),collapse=".",sep=""))
    ,collapse="_",sep="")
  return(im_fname)
}
prefix_from_pvol <- function(pvol){
  date_prefix <- strftime(pvol$datetime,format="%Y/%m/%d")
  corad <- pvol$radar
  co <- substring(corad,1,2)
  rad <- substring(corad,3,5)
  corad_prefix <- paste0(c(co,rad),collapse="/")
  corad_prefix <- toupper(corad_prefix)
  prefix <- paste0(c(corad_prefix,date_prefix),collapse="/")
  return(prefix)
}
title_from_pvol <- function(pvol){
  date_prefix <- strftime(pvol$datetime,format="%Y/%m/%d T %H:%M")
  corad <- pvol$radar
  co <- substring(corad,1,2)
  rad <- substring(corad,3,5)
  corad_prefix <- paste0(c(co,rad),collapse="/")
  corad_prefix <- toupper(corad_prefix)
  title_str <- paste0(c(co,rad,date_prefix),collapse=" ")
  title_str <- toupper(title_str)
  return(title_str)
}

dir.create(file.path(conf_local_visualization_output), showWarnings = FALSE)
idx=1
local_ppi_paths <- list()
for (pvol_path in local_pvol_paths) {
  my_pvol <- bioRad:::read_pvolfile(pvol_path)
  my_scan <- bioRad:::get_scan(x=my_pvol,
                               elev=param_elevation)
  my_ppi <- bioRad:::project_as_ppi(x=my_scan)
  image_filename <- im_fname_from_path(path=pvol_path,
                                       elev=param_elevation,
                                       param=param_param
  )
  local_prefix <- prefix_from_pvol(my_pvol)
  local_image_dir <- paste(conf_local_visualization_output,local_prefix,collapse="",sep="/")
  dir.create(file.path(local_image_dir),recursive=T,showWarnings = FALSE)
  local_image_path <- paste(local_image_dir,image_filename,collapse="",sep="/")
  print(local_image_path)
  png(local_image_path,width=1024,height=1024,units="px")
  title_str <- title_from_pvol(my_pvol)
  p <- plot(x=my_ppi,
       param=param_param) + ggtitle(title_str) + theme(plot.title = element_text(size = 40, face = "bold",hjust=0.5))
  print(p)
  dev.off()
  local_ppi_paths <- append(local_ppi_paths,local_image_path)
}

In [ ]:
# DO NOT CONTAINERISE
# parametrization - v95
# Run specific - fill in text to preset, keep empty to prompt



param_radar = "HRW"  # DHL | HRW | DBL

param_concurrency = 5
param_interval = 60
# This is to control uploads / cleaning
# Move results to S3?
param_upload_results = "True"
# Remove input after processing KNMI format to ODIM format
param_clean_knmi_input = "True"
# Should we remove the final Polar Volumes after producing a VP and or RBC
param_clean_pvol_output = "True"
#
param_clean_vp_output = "True"

# Param
### User specific, not neccesarily run specific.
#### Update: Perhaps some of these userinfo should be hardcoded parameters. I mean
#### Not something we'd enter every single time but should be set at setup time.
param_jupyterhub_user = 'bwijers1@gmail.com'

#### Visualization parameters
param_dtype = "pvol"  # pvol | vp
param_country = "NL"  # only NL
param_year = "2023"  # as string, YYYY
param_month = "12"  # as string, mm
param_day = "31"  # as string, dd

# Perhaps param prefix works better
param_prefix = "NL/DHL/2023/12/31"

# parameters
param_elevation = 3  #
param_param = "VRADH"  # I know...VRADH, DBZH, TH, WRADH, RHOHV, DBZ...
param_imtype = "ppi"  # Likely only type so far. PVOL -> PPI, VP -> VPTS. Future there would be more.

In [ ]:
# DO NOT CONTAINERISE
# configuration - v95


conf_minio_user_bucket_name = "naa-vre-user-data"  # the user bucket name
conf_minio_public_bucket_name = "naa-vre-public"  # the public bucket name

conf_minio_endpoint = "scruffy.lab.uvalight.net:9000"

### Directories
conf_local_root = "/tmp/data"
conf_local_knmi = "/tmp/data/knmi"
conf_local_odim = "/tmp/data/odim"
conf_local_vp = "/tmp/data/vp"
conf_local_ppi = "/tmp/data/ppi"
conf_local_vpts = "/tmp/data/vpts"
conf_local_conf = "/tmp/data/conf"
conf_local_radar_db = "/tmp/data/conf/OPERA_RADARS_DB.json"
conf_local_visualization_input = "/tmp/data/visualizations/input"
conf_local_visualization_output = "/tmp/data/visualizatons/output"

conf_pvol_output_prefix = "pvol"
conf_vp_output_prefix = "vp"
conf_ppi_output_prefix = "ppi"
conf_vpts_output_prefix = "vpts"

# radar configuration for the KNMI api
# Rewritten in a long format without page breaks. This is to prevent
# the code analyzer to yield an error.
# datasetName, datasetVersion, api_url, radar code (odim)


In [ ]:
# VP R Code
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - local_vp_paths : List
#   outputs:
#    - local_vpts_paths : List
# ...

# Libraries
library('bioRad')
library('ggplot2')

# Code analyzer fix block. Assign empty strings to variables that should not be picked up by the analyzer.
print(conf_local_visualization_output)

# Functions
im_fname_from_vpts <- function(regvpts){
  date_range = regvpts$daterange
  radar = regvpts$radar
  imtype = "regvpts"
  dr1 <- as.character(date_range[[1]])
  dr2 <- as.character(date_range[[2]])
  dr2 <- strsplit(dr2,split=" ")
  dr2 <- paste0(dr2[[1]],collapse="T")
  dr_str <- paste0(c(dr1,dr2),collapse="_")
  fname_noext <- paste0(c(radar,imtype,dr_str),collapse="_")
  im_fname <- paste0(fname_noext,".png")
  return (im_fname)
}
#  check if we do this on YYYY/MM instead of day, as vpts can cover more than 1 day.
prefix_from_vpts <- function(vpts){
  date_prefix <- strftime(vpts$datetime[[1]],format="%Y/%m") 
  corad <- vpts$radar
  co <- substring(corad,1,2)
  rad <- substring(corad,3,5)
  corad_prefix <- paste0(c(co,rad),collapse="/")
  corad_prefix <- toupper(corad_prefix)
  prefix <- paste0(c(corad_prefix,date_prefix),collapse="/")
  return (prefix)
}

title_str_from_regvpts <- function(regvpts){
  date_range = regvpts$daterange
  radar = regvpts$radar
  imtype = "regvpts"
  dr1 <- as.character(date_range[[1]])
  dr2 <- as.character(date_range[[2]])
  dr2 <- strsplit(dr2,split=" ")
  dr2 <- paste0(dr2[[1]],collapse="T")
  dr_str <- paste0(c(dr1,dr2),collapse="_")
  #fname_noext <- paste0(c(radar,imtype,dr_str),collapse="_")
  title_str <- paste0(c(radar,imtype,dr_str),collapse=" ")
  return (title_str)
}

# Load the VP files
print(local_vp_paths)
vp_list <- bioRad:::read_vpfiles(local_vp_paths)
vpts <- bioRad:::bind_into_vpts(vp_list)
reg_vpts <- bioRad:::regularize_vpts(vpts)
# determine local image path
image_filename = im_fname_from_vpts(reg_vpts)
local_prefix <- prefix_from_vpts(vpts)
local_image_dir <- paste(conf_local_visualization_output,local_prefix,collapse="",sep="/")
dir.create(local_image_dir,showWarnings=F,recursive=T)
local_image_path <- paste(local_image_dir,image_filename,collapse="",sep="/")
print(local_image_path)
title_str <- title_from_pvol(my_pvol) 
png(local_image_path,width=1024,height=1024,units="px") 
p <- plot(reg_vpts) 
title(title_str,line=3)
print(p)
dev.off()
local_vpts_paths <- list(local_image_path)

In [ ]:
# R-PPI
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - local_pvol_paths : List
#   outputs:
#    - local_ppi_paths : List
#   params:
#    - param_param : String
#    - param_elevation : Float
# ...

# libraries
library('bioRad')
library("stringr")
library("ggplot2")
title_from_pvol <- function(pvol) {
  date_prefix <- strftime(my_pvol$datetime, format = "%Y/%m/%d T %H:%M UTC", tz = "UTC")
  corad <- pvol$radar
  co <- substring(corad, 1, 2)
  rad <- substring(corad, 3, 5)
  corad_prefix <- paste0(c(co, rad), collapse = "/")
  corad_prefix <- toupper(corad_prefix)
  title_str <- paste0(c(co, rad, date_prefix), collapse = " ")
  title_str <- toupper(title_str)
  return(title_str)
}
# Retrieve the basemap
basemap <- rosm::osm.types()[1]
# initialize a output list for files
local_ppi_paths <- list()
for (pvol_path in local_pvol_paths) {
  my_pvol <- read_pvolfile(pvol_path, param = c("DBZH", "VRADH"))
  my_scan <- get_scan(
    x = my_pvol,
    elev = param_elevation,
    all = FALSE
  )
  my_param <- get_param(my_scan, param = param_param)
  ppi <- project_as_ppi(my_param)
  # If elevation has a '.' in it, like 2.5, substitute the '.' for a '-'.
  # Otherwise certain operating systems confuse it for a suffix.
  elev <- my_scan$attributes$where$elangle %>% str_replace("\\.", "-")
  # Create an image name
  imname <- basename(pvol_path) %>%
    str_replace(".h5", ".png") %>%
    str_replace("pvol", paste("ppi", param_param, elev, sep = "_"))
  print(imname)
  # Create an image directory
  impath_dir <- pvol_path %>%
    str_replace(conf_local_odim, conf_local_ppi) %>%
    str_replace(basename(pvol_path), "")
  impath_dir <- paste(impath_dir, param_param, elev, sep = "/")
  # Create the directory
  dir.create(file.path(impath_dir), recursive = TRUE, showWarnings = FALSE)
  # Create a image path
  impath <- paste(impath_dir, imname, sep = "/")
  title_str <- title_from_pvol(my_pvol)  
  subtitle_str <- paste("param:",param_param, "- elevation angle:",param_elevation)
  png(impath, width = 1024, height = 1024, units = "px")
  plt <- map(ppi, map = basemap, alpha = 0.6) +
    ggtitle(title_str, subtitle = subtitle_str) +
    theme(plot.title = element_text(size = 40, face = "bold", hjust = 0.5), plot.subtitle = element_text(hjust = 0.5))
  print(plt)
  dev.off()
  local_ppi_paths <- append(local_ppi_paths, impath)
}